In [26]:
%matplotlib notebook
%load_ext autoreload
%pwd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'/ocean/projects/asc170022p/mtragoza/lung-project/notebooks'

In [2]:
import sys
import pandas as pd
sys.path.append('../../param_search')
import param_search as ps

In [22]:
# define a job template and name format
template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:1
#SBATCH -x v034
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
module load anaconda3
conda activate /ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project
nvidia-smi

data_root=../../../data/{data_name}

case_name=Case{case_id:d}{case_suffix}
case_dir=$data_root/$case_name

image_dir=$case_dir/NIFTI
mask_dir=$case_dir/TotalSegment

image_name=case{case_id:d}_T{phase:02d}
image_file=$image_dir/$image_name.nii.gz
output_dir=$mask_dir/$image_name

TotalSegmentator -i $image_file -o $output_dir -d gpu -ta total --roi_subset lung_upper_lobe_right lung_upper_lobe_left lung_middle_lobe_right lung_lower_lobe_right lung_lower_lobe_left
TotalSegmentator -i $image_file -o $output_dir -d gpu -ta lung_vessels
TotalSegmentator -i $image_file -o $output_dir -d gpu -ta body
totalseg_combine_masks -i $output_dir -o $output_dir/lung_combined_mask.nii.gz -m lung

echo Done
'''
name_format = 'segment_{data_name}_case{case_id:d}_T{phase:02d}'

In [23]:
param_space = []
for p in ps.ParamSpace(
    data_name='Emory-4DCT',
    case_id=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    phase=[0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
):
    if p['case_id'] == 8:
        p['case_suffix'] = 'Deploy'
    else:
        p['case_suffix'] = 'Pack'
    param_space.append(p)

for p in param_space:
    print(name_format.format(**p))

segment_Emory-4DCT_case1_T00
segment_Emory-4DCT_case1_T10
segment_Emory-4DCT_case1_T20
segment_Emory-4DCT_case1_T30
segment_Emory-4DCT_case1_T40
segment_Emory-4DCT_case1_T50
segment_Emory-4DCT_case1_T60
segment_Emory-4DCT_case1_T70
segment_Emory-4DCT_case1_T80
segment_Emory-4DCT_case1_T90
segment_Emory-4DCT_case2_T00
segment_Emory-4DCT_case2_T10
segment_Emory-4DCT_case2_T20
segment_Emory-4DCT_case2_T30
segment_Emory-4DCT_case2_T40
segment_Emory-4DCT_case2_T50
segment_Emory-4DCT_case2_T60
segment_Emory-4DCT_case2_T70
segment_Emory-4DCT_case2_T80
segment_Emory-4DCT_case2_T90
segment_Emory-4DCT_case3_T00
segment_Emory-4DCT_case3_T10
segment_Emory-4DCT_case3_T20
segment_Emory-4DCT_case3_T30
segment_Emory-4DCT_case3_T40
segment_Emory-4DCT_case3_T50
segment_Emory-4DCT_case3_T60
segment_Emory-4DCT_case3_T70
segment_Emory-4DCT_case3_T80
segment_Emory-4DCT_case3_T90
segment_Emory-4DCT_case4_T00
segment_Emory-4DCT_case4_T10
segment_Emory-4DCT_case4_T20
segment_Emory-4DCT_case4_T30
segment_Emory-

In [24]:
jobs = ps.submit(template, name_format, param_space, work_dir='2024-06-28_segment')
jobs

100%|██████████| 100/100 [00:03<00:00, 30.28it/s]


/ocean/projects/asc170022p/mtragoza/lung-project/notebooks/../../param_search/param_search/job_queues.py:235: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace('', float('nan')).map(pd.to_numeric)


,data_name,case_id,phase,case_suffix,job_name,job_id,partition,job_state,node_id,runtime,work_dir,array_idx
0,Emory-4DCT,1,0,Pack,segment_Emory-4DCT_case1_T00,24356758,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
1,Emory-4DCT,1,10,Pack,segment_Emory-4DCT_case1_T10,24356759,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
2,Emory-4DCT,1,20,Pack,segment_Emory-4DCT_case1_T20,24356760,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
3,Emory-4DCT,1,30,Pack,segment_Emory-4DCT_case1_T30,24356761,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
4,Emory-4DCT,1,40,Pack,segment_Emory-4DCT_case1_T40,24356762,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
95,Emory-4DCT,10,50,Pack,segment_Emory-4DCT_case10_T50,24356854,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
96,Emory-4DCT,10,60,Pack,segment_Emory-4DCT_case10_T60,24356855,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
97,Emory-4DCT,10,70,Pack,segment_Emory-4DCT_case10_T70,24356856,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN
98,Emory-4DCT,10,80,Pack,segment_Emory-4DCT_case10_T80,24356857,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN


In [25]:
status = ps.status(jobs)
status

/ocean/projects/asc170022p/mtragoza/lung-project/notebooks/../../param_search/param_search/job_queues.py:235: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .replace('', float('nan')).map(pd.to_numeric)
/ocean/projects/asc170022p/mtragoza/lung-project/notebooks/../../param_search/param_search/__init__.py:80: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING'
 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING'
 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING'
 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING'
 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDING' 'PENDI

,index,data_name,case_id,phase,case_suffix,job_name,partition,job_state,node_id,runtime,work_dir,array_idx,stdout,stderr
job_id,,,,,,,,,,,,,,
24356758,0,Emory-4DCT,1,0,Pack,segment_Emory-4DCT_case1_T00,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
24356759,1,Emory-4DCT,1,10,Pack,segment_Emory-4DCT_case1_T10,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
24356760,2,Emory-4DCT,1,20,Pack,segment_Emory-4DCT_case1_T20,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
24356761,3,Emory-4DCT,1,30,Pack,segment_Emory-4DCT_case1_T30,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
24356762,4,Emory-4DCT,1,40,Pack,segment_Emory-4DCT_case1_T40,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24356854,95,Emory-4DCT,10,50,Pack,segment_Emory-4DCT_case10_T50,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
24356855,96,Emory-4DCT,10,60,Pack,segment_Emory-4DCT_case10_T60,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN
24356856,97,Emory-4DCT,10,70,Pack,segment_Emory-4DCT_case10_T70,GPU-shared,PENDING,(None),0:00,/ocean/projects/asc170022p/mtragoza/lung-proje...,NaN,NaN,NaN


In [13]:
print(status.stdout.iloc[-10])

nan


In [22]:
import sys, os
os.environ['PKG_CONFIG_PATH'] = '/ocean/projects/asc170022p/mtragoza/mambaforge/envs/lung-project/lib/pkgconfig'
sys.path.append('..')
import mycode as code
import hvplot.xarray

AttributeError: module 'fenics' has no attribute 'cpp'

In [ ]:
%autoreload
case = code.data.Emory4DCTCase('data/Emory-4DCT', 'Case1Pack', [0, 50])
case.load_niftis()
case.load_masks()

In [ ]:
data.view_array(case.array, groupby=['phase', 'y'], x='x', y='z')

In [ ]:
case.describe()

In [ ]:
import vedo
vedo.settings.default_backend= 'ipyvtklink'
import visual

In [ ]:
%autoreload

visual.IsosurfaceBrowser(
    ds.array,
    isovalue=845,
    c='white',
    alpha=0.5,
    use_gpu=True
)

In [ ]:
help(vedo.Volume)